<a href="https://colab.research.google.com/github/hellozhaojian/transformers/blob/master/Pre_training_Bert_With_wiki_zh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**简介**

该文档描述了如何从零开始用TPU训练模型。
前提假设：

1. 有一个google cloud账号， 创建一个项目；创建一个bucket
2. 有google drive账号

操作步骤：
  1. 安装必要软件。主要安装sentencepiece和bert
  2. 测试运行环境。
  3. 准备语料。将google drive 上的数据挂载到当前的运行环境。
  4. 准备wordpiece词典。 并将结果转换为wordpiece需要的格式（支持中文)
  5. 训练环境准备。
  
     准备训练数据。执行bert里的数据准的脚本生成tf-record格式的数据。
     将tf-record，wordpiece词典，以及bert的配置文件拷贝的google cloud的文件系统中。
  6. TPU训练。
  7. 拷贝预训练模型。



**一、安装必要的软件**




In [1]:
!pip install sentencepiece
!git clone https://github.com/google-research/bert

fatal: destination path 'bert' already exists and is not an empty directory.


**二、测试运行环境**


In [2]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder
print(tf.__version__)



1.15.0


**三、准备语料**

* 挂载google drive


In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')

dataset_file = "/content/drive/My Drive/data/wiki.txt"
bert_meta_dir = "/content/drive/My Drive/bert/"
! mkdir -p bert_meta_dir

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* 语料截断

In [5]:
DEMO_MODE = True #@param {type:"boolean"}

if DEMO_MODE:
  CORPUS_SIZE = 100000
else:
  CORPUS_SIZE = 100000000 #@param {type: "integer"}
! wc -l "/content/drive/My Drive/data/wiki.txt"
! (head -n $CORPUS_SIZE "/content/drive/My Drive/data/wiki.txt") > dataset.txt
! head dataset.txt
! wc -l dataset.txt

5249001 /content/drive/My Drive/data/wiki.txt
燕顺，是《水浒传》人物，山东莱州人氏，赤色黄发，人称「锦毛虎」。原是羊马商人出身，后来本钱用光，便流落至清风山打劫，为清风山贼王，手下有王英及郑天寿。
一次，宋江路过清风山，被山贼绑了，押往寨主燕顺处。燕顺的喽啰正要剖开宋江的心肝，宋江悲叹大喊「想不到我宋江竟要命丧于此!」，燕顺闻得宋江名号便下令叫停，领自己的手下一同向宋江下拜。
宋江在清风山救了清风寨知寨刘高的妻子，但她恩将仇报，要刘高抓了宋江。当黄信押着宋江和花荣途经清风山时，燕顺等人前往劫囚，救出二人。然后，清风山一伙连同花荣与秦明都上了梁山泊。
燕顺为马军小彪将兼远探出哨头领第十一员，后来随征辽国、方腊，在乌龙岭被方腊大将石宝的流星锤打死。
六里屯街道
六里屯街道是北京市朝阳区东部的一个街道办事处，前身为星火人民公社，辖区内的星火站因此得名（现属东风地区）。
始建于1987年。东起双沙铁路，南至二道沟河，西邻团结湖街道，北到农展南路。面积4.4平方公里，常住人口8.9万。
六里屯街道目前下辖的社区有：
奇幻文学
奇幻文学是指文学类的奇幻作品，主要表达于奇幻小说。广义上含有奇幻要素的文学作品也包括在内，玄幻小说、武侠小说与古代流传的童话、神话、民间传奇故事。
100000 dataset.txt


**四、准备wordpiece的词典**


* 训练sentencepiece

In [6]:
PRC_DATA_FPATH = "dataset.txt"
MODEL_PREFIX = "tokenizer" #@param {type: "string"}
VOC_SIZE = 32000 #@param {type:"integer"}
SUBSAMPLE_SIZE = 12800000 #@param {type:"integer"}
NUM_PLACEHOLDERS = 256 #@param {type:"integer"}

SPM_COMMAND = ('--input={} --model_prefix={} '
               '--vocab_size={} --input_sentence_size={} '
               '--shuffle_input_sentence=true ' 
               '--bos_id=-1 --eos_id=-1').format(
               PRC_DATA_FPATH, MODEL_PREFIX, 
               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)
print(SPM_COMMAND)
spm.SentencePieceTrainer.Train(SPM_COMMAND)



--input=dataset.txt --model_prefix=tokenizer --vocab_size=31744 --input_sentence_size=12800000 --shuffle_input_sentence=true --bos_id=-1 --eos_id=-1


True

* 测试sentencepiece

In [7]:
sp = spm.SentencePieceProcessor()
sp.load('{}.model'.format(MODEL_PREFIX))
token_list =  sp.encode_as_pieces("这是一个测试，你要看看吗")
print(token_list)
! ls

['▁', '这是一个', '测试', ',', '你', '要', '看', '看', '吗']
adc.json       bert_model   pretraining_data  tokenizer.model
bert	       dataset.txt  sample_data       tokenizer.vocab
bert_meta_dir  drive	    shards	      vocab.txt


* 将sentencepiece的词典 修正为wordpiece对应的词典


In [0]:
import unicodedata

def _is_whitespace(char):
  """Checks whether `chars` is a whitespace character."""
  # \t, \n, and \r are technically contorl characters but we treat them
  # as whitespace since they are generally considered as such.
  if char == " " or char == "\t" or char == "\n" or char == "\r":
    return True
  cat = unicodedata.category(char)
  if cat == "Zs":
    return True
  return False


def _is_control(char):
  """Checks whether `chars` is a control character."""
  # These are technically control characters but we count them as whitespace
  # characters.
  if char == "\t" or char == "\n" or char == "\r":
    return False
  cat = unicodedata.category(char)
  if cat in ("Cc", "Cf"):
    return True
  return False


def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False



In [9]:
def parse_sentencepiece_token(token) :
    """
    sentencepiece用特殊符号 "▁"截断字符串，而wordpiece用的是 ##, 该函数将sentencepiece学习出来的词映射为woedpiece的词。
    """
    
    def _is_chinese_char( cp):
        """Checks whether CP is the codepoint of a CJK character."""
        # This defines a "chinese character" as anything in the CJK Unicode block:
        #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
        #
        # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
        # despite its name. The modern Korean Hangul alphabet is a different block,
        # as is Japanese Hiragana and Katakana. Those alphabets are used to write
        # space-separated words, so they are not treated specially and handled
        # like the all of the other languages.
        if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
            (cp >= 0x3400 and cp <= 0x4DBF) or  #
            (cp >= 0x20000 and cp <= 0x2A6DF) or  #
            (cp >= 0x2A700 and cp <= 0x2B73F) or  #
            (cp >= 0x2B740 and cp <= 0x2B81F) or  #
            (cp >= 0x2B820 and cp <= 0x2CEAF) or
            (cp >= 0xF900 and cp <= 0xFAFF) or  #
            (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
            return True
    
    if token.startswith('▁'):
        return token[1:]
    if len(list(token)) == 1 and (_is_chinese_char(ord(token)) or _is_punctuation(token) or _is_control(token)  or _is_whitespace(token)):
        return token
    else:
        return  "##" + token
    
def strQ2B(ustring):
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return "".join(ss)

print(list(map(parse_sentencepiece_token, token_list)))


['', '##这是一个', '##测试', ',', '你', '要', '看', '看', '吗']


In [0]:
VOC_FNAME = "vocab.txt" #@param {type:"string"}

voc_file = "{}.vocab".format(MODEL_PREFIX)

def read_sentencepiece_vocab(file_path):
    voc = []
    for line in open(file_path, "r"):
        voc.append(line.strip().split("\t")[0])
    voc = voc[1:]
    return voc
voc_words = read_sentencepiece_vocab(voc_file)

new_voc_words = list(map(parse_sentencepiece_token, voc_words))

ctrl_symbols =  ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
bert_words = ctrl_symbols +  new_voc_words

bert_words += ["[UNUSED_{}]".format(i) for i in range(VOC_SIZE - len(bert_words))]


fo =  open(VOC_FNAME, "w", encoding='utf-8')
for token in bert_words:
    fo.write(token + "\n")
fo.close()
! head -n 20 $VOC_FNAME
! tail -n 10 $VOC_FNAME
bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)

print(bert_tokenizer.tokenize(strQ2B("这是一个测试，你要看看吗, what are you doing here")))
print(bert_tokenizer.tokenize("Colorless geothermal substations are generating furiously"))

***五、训练环境准备***


* 准备tf-record文件


In [16]:
! rm -rf shards
! mkdir ./shards
# 将文件切成多分，每一份最多250000行。 新的文件后缀用数字表示(-d), 用4位数字(-a 4)
! split -a 4 -l 250000 -d dataset.txt ./shards/shard_d
! ls ./shards/
! wc -l ./shards/*
! du -sm *

shard_d0000
100000 ./shards/shard_d0000
1	adc.json
1	bert
1	bert_meta_dir
1	bert_model
26	dataset.txt
1167	drive
214	pretraining_data
55	sample_data
26	shards
1	tokenizer.model
1	tokenizer.vocab
1	vocab.txt


In [0]:
MAX_SEQ_LENGTH = 128 #@param {type : "integer"}
MASKED_LM_PROB = 0.15 #@param {type: "number" }
# xxxx
MAX_PREDICTIONS = 20 #@param {type: "integer"}
DO_LOWER_CASE = True #@param {type: "boolean"}
PROCESSES = 5 #@param {type: "integer"}
PRETRAINING_DIR = "pretraining_data" #@param {type: "string"}
GOOGLE_CLOUD_PROJECT_NAME = "pre-train-bert-sogou" #@param {type: "string"}
BUCKET_NAME = "bert-sogou-pretrain"  #@param {type: "string"}

MODEL_DIR = "bert_model" #@param {type: "string"}
tf.gfile.MkDir(MODEL_DIR)

In [0]:

XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

print (XARGS_CMD)
tf.gfile.MkDir(PRETRAINING_DIR)
! $XARGS_CMD


* 将bert相关的配置和数据上传到bucket里

In [0]:

# use this for BERT-base

bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": VOC_SIZE
}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
  json.dump(bert_base_config, fo, indent=2)
  
with open("{}/{}".format(MODEL_DIR, VOC_FNAME), "w") as fo:
  for token in bert_words:
    fo.write(token+"\n")

! gcloud auth login
! gcloud config set project $GOOGLE_CLOUD_PROJECT_NAME
! gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME


**六、TPU训练**

* 链接TPU

前提条件： 在项目$GOOGLE_CLOUD_PROJECT_NAME中已经申请了TPU


In [34]:
import logging
log = logging.getLogger("pre-train-bert")
auth.authenticate_user()

if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(TPU_ADDRESS) as session:
    print(TPU_ADDRESS)
    log.info('TPU address is ' + TPU_ADDRESS)
    # Upload credentials to TPU.
    #with open('/content/adc.json', 'r') as f:
    #  auth_info = json.load(f)
    #tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    tf.contrib.cloud.configure_gcs(session)
else:
  log.warning('Not connected to TPU runtime')
  USE_TPU = False
print(USE_TPU)




grpc://10.62.221.2:8470
True


* 设置训练参数

In [0]:

# Input data pipeline config
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"}
MAX_PREDICTIONS = 20 #@param {type:"integer"}
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 1000 #@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 25 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME:
  BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
  BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.gfile.Glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))



* 训练

In [0]:
print(bert_config)

model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

INFO:tensorflow:Using config: {'_model_dir': 'gs://bert-sogou-pretrain/bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 25, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.62.221.2:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f51e3d41438>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.62.221.2:8470', '_evaluation_master': 'grpc://10.62.221.2:8470', '_is_chief': True, '_num_ps_replicas'

**七、拷贝预训练的模型**


模型存储在 $MODEL_DIR 下面。用户需要拷贝 vocb.txt, bert_config.json, 以及训练的模型文件。
